<a href="https://colab.research.google.com/github/aakarshhh/AI_ML/blob/main/Sent_Stock_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install praw  textblob  nltk yfinance

In [247]:
import praw
import csv
import re
import json   
import requests
import time
import numpy as np
import pandas as pd
from textblob import TextBlob
import datetime
from re import search
# VADER can be accessed by the NLTK library.
import nltk
from urllib.request import urlopen
# Download the VADAR tool and access it through the NLTK library.
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download("vader_lexicon")
from nltk.sentiment.vader import SentimentIntensityAnalyzer as sia

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [269]:
def get_jsonparsed_data(url):
    response = urlopen(url)
    data = response.read().decode("utf-8")
    return json.loads(data)
company = pd.read_excel('/content/Companies.xlsx',header=None)    
ticker = pd.read_excel('/content/tickers.xlsx',header=None)
ticker = pd.DataFrame({'Ticker':ticker[0],'Company':company[0]})
ticker['Buy']=0
ticker['Hold']=0
ticker['Sell']=0
#for w in data[0] :
#  url = (f"https://financialmodelingprep.com/api/v3/search?query={w.strip()}&limit=1&apikey=demo")
#  d = ticker.append([get_jsonparsed_data(url)
#  ticker = ticker.append([get_jsonparsed_data(url)[0]['symbol']]) 

In [249]:
a = 'PG'
url = (f"https://financialmodelingprep.com/api/v3/search?query={a}&limit=1&apikey=demo")
get_jsonparsed_data(url)[0]['symbol']

'PG'

In [270]:
reddit = praw.Reddit( client_secret = 'BnSem-azR1PT-KOweDLrTDx_GoOOYA',
 client_id = 'bpxOQpC_a3i8xA',
 user_agent = 'StonksSent',
 username = 'LaLaA80524',
 password = 'SentStock')

In [271]:
subr= reddit.subreddit('wallstreetbets+robinhood+algotrading+investing+stocks+SecurityAnalysis+StockMarket')

In [272]:
t = datetime.datetime.now()
t = t- datetime.timedelta(days = 7)
print(t)

2021-03-27 05:18:46.459879


In [273]:
top_stuff=subr.top(time_filter='week' ,limit=500)
for submission in top_stuff:
  submission.comments.replace_more(limit=0)
  comments = submission.comments.list() 
  for word in word_tokenize(submission.title):
    for i in range(len(ticker)):
      if word == '$'+ ticker['Ticker'][i] or word == ticker['Ticker'][i] or str(ticker['Company'][i]).find(word)!=-1:
        buy_upvote = 0 
        sell_upvote = 0 
        hold_upvote = 0 
        comments = submission.comments.list()
        for comment in comments :
         if comment.created_utc >= t.replace(tzinfo=datetime.timezone.utc).timestamp():
           buy_list = re.findall(r'\bbuy\b',comment.body, flags = re.IGNORECASE )
           sell_list = re.findall(r'\bsell\b',comment.body, flags = re.IGNORECASE )
           hold_list = re.findall(r'\bhold\b',comment.body, flags = re.IGNORECASE )
           d_buy_list = re.findall(r"\bdon't buy\b",comment.body, flags = re.IGNORECASE )
           d_sell_list = re.findall(r"\bdon't sell\b",comment.body, flags = re.IGNORECASE )
           d_hold_list = re.findall(r"\bdon't hold\b",comment.body, flags = re.IGNORECASE )
           if d_buy_list ==[]:
             if buy_list != []:
               buy_upvote +=comment.ups
           if sell_list == []:
             _
           elif d_sell_list == []:
             sell_upvote +=comment.ups
           else:
             hold_upvote+=comment.ups
    
           if hold_list == []:
             _
           elif d_hold_list == []:
             hold_upvote+=comment.ups
           else:
             sell_upvote += comment.ups
        ticker['Buy'][i]+=buy_upvote/len(word_tokenize(ticker['Company'][i]))
        ticker['Hold'][i]+=hold_upvote/len(word_tokenize(ticker['Company'][i]))
        ticker['Sell'][i]+=sell_upvote/len(word_tokenize(ticker['Company'][i]))
t = datetime.datetime.now()

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy 

In [276]:
ticker[25:]

,Ticker,Company,Buy,Hold,Sell
25,EQIX,Equinix,469,156,643
26,SPG,Simon Property group,691,403,309
27,PSA,Public Storage,2747,808,1210
28,WELL,Welltower,1891,1464,1434
29,EQR,Equity Residential,1735,117,567
30,AVB,AvalonBay Communities,2090,256,694
31,SBAC,SBA Communication,3518,232,1181
32,XOM,ExxonMobil,715,306,375
33,CVX,Chevron,708,284,260
34,NEP,NextEra Energy,1598,58,403
